# Task 2: Bayesian Change Point Analysis and Insight Generation

This notebook implements Bayesian change point detection using PyMC3 to identify structural breaks in Brent oil price time series and correlate them with geopolitical events.

## Objectives:
1. **Implement Bayesian Change Point Models** using PyMC3
2. **Detect Structural Breaks** in oil price time series
3. **Correlate Change Points** with compiled events
4. **Quantify Event Impacts** with statistical measures
5. **Generate Insights** for stakeholders

In [1]:
# Standard data science imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Bayesian modeling
import pymc as pm
import arviz as az

# Statistical analysis
from scipy import stats
from scipy.stats import norm, t

# Time series utilities
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Import our custom modules
import sys
sys.path.append('../src')
from analysis.change_point import BayesianChangePointAnalysis
from analysis.event_research import EventResearch

print("Setup complete!")
print(f"PyMC version: {pm.__version__}")
print(f"ArviZ version: {az.__version__}")

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


Setup complete!
PyMC version: 5.25.1
ArviZ version: 0.22.0


In [2]:
# Load processed data from Task 1
data = pd.read_csv('../data/processed/brent_oil_prices_processed.csv', index_col=0)
data.index = pd.to_datetime(data.index)

# Load event data
events = pd.read_csv('../data/processed/major_events.csv')
events['date'] = pd.to_datetime(events['date'])

print("Data loaded successfully!")
print(f"Price data shape: {data.shape}")
print(f"Events data shape: {events.shape}")
print(f"Date range: {data.index.min()} to {data.index.max()}")
print(f"Number of events: {len(events)}")

Data loaded successfully!
Price data shape: (9011, 5)
Events data shape: (29, 9)
Date range: 1987-05-20 00:00:00 to 2022-11-14 00:00:00
Number of events: 29


In [3]:
# Calculate log returns for change point analysis
data['log_returns'] = np.log(data['price'] / data['price'].shift(1))
data['price_change'] = data['price'].pct_change()

# Remove NaN values
data_clean = data.dropna()

print("Data preprocessing complete!")
print(f"Clean data shape: {data_clean.shape}")
print(f"Log returns statistics:")
print(data_clean['log_returns'].describe())

Data preprocessing complete!
Clean data shape: (8981, 5)
Log returns statistics:
count    8981.000000
mean        0.000178
std         0.025571
min        -0.643699
25%        -0.011222
50%         0.000395
75%         0.012197
max         0.412023
Name: log_returns, dtype: float64


In [4]:
# Initialize the change point analysis
cpa = BayesianChangePointAnalysis(data_clean, events)

print("Bayesian Change Point Analysis initialized!")
print(f"Data points: {len(cpa.log_returns)}")
print(f"Events: {len(events)}")

# Display log returns statistics
print(f"\nLog returns statistics:")
print(f"Mean: {cpa.log_returns.mean():.6f}")
print(f"Std: {cpa.log_returns.std():.6f}")
print(f"Min: {cpa.log_returns.min():.6f}")
print(f"Max: {cpa.log_returns.max():.6f}")

Bayesian Change Point Analysis initialized!
Data points: 8980
Events: 29

Log returns statistics:
Mean: 0.000176
Std: 0.025572
Min: -0.643699
Max: 0.412023


In [5]:
# Implement and fit single change point model
print("Implementing single change point model...")

single_model = cpa.single_change_point_model(n_changepoints=1)

# Fit the model
print("Fitting single change point model...")
single_results = cpa.fit_model(single_model, 'single_change_point', n_samples=2000, n_tune=1000)

print("\nSingle change point model fitted successfully!")
print("\nModel summary:")
print(single_results['summary'])

# Analyze results
single_analysis = cpa.analyze_change_points('single_change_point')

print("\nSingle Change Point Analysis Results:")
print(f"Detected change points: {single_analysis['change_points']}")
print(f"Number of event correlations: {len(single_analysis['event_correlations'])}")
print(f"Number of impact measures: {len(single_analysis['impact_analysis'])}")

Implementing single change point model...


TypeError: TensorVariable cannot be converted to Python integer. Call `.astype(int)` for the symbolic equivalent.

In [9]:
# Implement and fit event-correlated model
print("Implementing event-correlated change point model...")

event_model = cpa.event_correlated_model()

# Fit the model
print("Fitting event-correlated model...")
event_results = cpa.fit_model(event_model, 'event_correlated', n_samples=2000, n_tune=1000)

print("\nEvent-correlated model fitted successfully!")
print("\nModel summary:")
print(event_results['summary'])

# Analyze results
event_analysis = cpa.analyze_change_points('event_correlated')

print("\nEvent-Correlated Analysis Results:")
print(f"Detected change points: {event_analysis['change_points']}")
print(f"Number of event correlations: {len(event_analysis['event_correlations'])}")
print(f"Number of impact measures: {len(event_analysis['impact_analysis'])}")

Implementing event-correlated change point model...


NameError: name 'cpa' is not defined

In [ ]:
# Create comprehensive visualizations for each model
print("Generating change point visualizations...")

models = ['single_change_point', 'event_correlated']

for model_name in models:
    if model_name in cpa.results:
        print(f"\nPlotting results for {model_name}...")
        cpa.plot_results(model_name, f'../docs/{model_name}_analysis.png')

# Create interactive visualization
fig = go.Figure()

# Add price line
fig.add_trace(go.Scatter(
    x=data_clean.index,
    y=data_clean['price'],
    mode='lines',
    name='Brent Oil Price',
    line=dict(color='blue', width=1)
))

# Add change points from different models
colors = ['red', 'green']
for i, model_name in enumerate(models):
    if model_name in cpa.results:
        analysis = cpa.analyze_change_points(model_name)
        for cp in analysis['change_points']:
            fig.add_vline(x=cp, line_dash="dash", line_color=colors[i], 
                         annotation_text=f"{model_name.replace('_', ' ').title()}")

fig.update_layout(
    title='Brent Oil Prices with Detected Change Points',
    xaxis_title='Date',
    yaxis_title='Price (USD/barrel)',
    height=600
)
fig.show()

In [ ]:
# Generate key insights and business implications
print("Key Insights and Business Implications:")
print("=" * 60)

# Collect all results
all_change_points = []
all_correlations = []
all_impacts = []

for model_name in models:
    if model_name in cpa.results:
        analysis = cpa.analyze_change_points(model_name)
        all_change_points.extend(analysis['change_points'])
        all_correlations.extend(analysis['event_correlations'])
        all_impacts.extend(analysis['impact_analysis'])

# Remove duplicates
unique_change_points = list(set(all_change_points))
unique_change_points.sort()

print(f"\n1. CHANGE POINT DETECTION:")
print(f"   - Total unique change points detected: {len(unique_change_points)}")
print(f"   - Change point dates: {unique_change_points}")

if all_correlations:
    corr_df = pd.DataFrame(all_correlations)
    
    print(f"\n2. EVENT CORRELATION:")
    print(f"   - Total event correlations: {len(corr_df)}")
    print(f"   - Average distance to events: {corr_df['distance_days'].mean():.1f} days")
    print(f"   - Events within 7 days: {len(corr_df[corr_df['distance_days'] <= 7])}")
    print(f"   - Events within 30 days: {len(corr_df[corr_df['distance_days'] <= 30])}")

if all_impacts:
    impact_df = pd.DataFrame(all_impacts)
    
    print(f"\n3. IMPACT QUANTIFICATION:")
    print(f"   - Average price change: {impact_df['price_change'].mean():.2f}%")
    print(f"   - Average volatility change: {impact_df['volatility_change'].mean():.2f}%")
    print(f"   - Largest price increase: {impact_df['price_change'].max():.2f}%")
    print(f"   - Largest price decrease: {impact_df['price_change'].min():.2f}%")
    
    # Statistical significance
    t_stat, p_value = stats.ttest_1samp(impact_df['price_change'], 0)
    significance = "significant" if p_value < 0.05 else "not significant"
    print(f"   - Price changes are {significance} (p={p_value:.6f})")

print(f"\n4. BUSINESS IMPLICATIONS:")
print(f"   - Risk Management: Change points indicate regime changes requiring strategy adjustment")
print(f"   - Trading Strategies: Optimal entry/exit timing around detected change points")
print(f"   - Policy Planning: Understanding market stability periods for energy security")
print(f"   - Investment Timing: Structural breaks provide opportunities for portfolio rebalancing")

In [ ]:
# Save all results for further analysis
print("Saving results...")

# Collect all results
task2_results = {}

for model_name in models:
    if model_name in cpa.results:
        analysis = cpa.analyze_change_points(model_name)
        task2_results[model_name] = {
            'change_points': [cp.strftime('%Y-%m-%d') for cp in analysis['change_points']],
            'event_correlations': analysis['event_correlations'],
            'impact_analysis': analysis['impact_analysis'],
            'model_diagnostics': cpa.results[model_name]['diagnostics']
        }

# Save to files
import json
from datetime import datetime

# Save as JSON
with open('../data/processed/task2_change_point_results.json', 'w') as f:
    json.dump(task2_results, f, indent=2, default=str)

print("Results saved successfully!")
print(f"- JSON results: ../data/processed/task2_change_point_results.json")

print("\nTask 2: Change Point Modeling and Insight Generation - COMPLETE!")
print("=" * 70)
print(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Models implemented: {len(models)}")
print(f"Total change points detected: {len(unique_change_points)}")
print(f"Total event correlations: {len(all_correlations)}")
print(f"Total impact measures: {len(all_impacts)}")
print("Ready for Task 3: Interactive Dashboard Development!")